In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung.csv?fbclid=IwAR3S1fY9KCp7smHyYi_T2P0EGm_K2FA29Uleb-WGWJ3RjDpI1QZiQ4Z45b4',header=None)
df

In [ ]:
df_clean = df
y = df_clean[0].values
X = df_clean.drop([0],axis = 1).values

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res,y_res = sm.fit_resample(X,y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15,random_state=0)
from sklearn.decomposition import PCA
fr = PCA(n_components=50)
fr.fit(X_train)
X_train = fr.transform(X_train)
X_test = fr.transform(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(random_state=1)
clf2 = SVC(probability=True)
clf3 = (ExtraTreesClassifier(random_state=1))
eclf = StackingClassifier(
estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],stack_method = 'predict_proba',final_estimator=SVC(probability=True))
params = {'lr__n_estimators': [10,50,100], 'rf__gamma': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4]}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X_train_norm,y_train)
print("Best params : " + str(grid.best_params_))
print("10CV accuracy : "+str(grid.best_score_*100))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = grid.predict(X_test_norm) # ทดสอบ
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung_test.csv?fbclid=IwAR0Ecj02m1m6yXVNTanYUY8rQNlfvYgV6meCQaJbJvh5mKKT_r1lvD7d-Eo',header=None)

In [ ]:
df_clean = df
yt = df_clean[0].values
Xt = df_clean.drop([0],axis = 1).values

In [ ]:
X_test = fr.transform(Xt)

In [ ]:
X_test_norm = scaler.transform(X_test)

In [ ]:
y_predict = grid.predict(X_test_norm) # ทดสอบ
print("Test accuracy : "+str(sum(yt == y_predict)/len(yt)*100))